In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pylab as plt
%reload_ext autoreload
%autoreload 2

plt.rcParams.update({
    "text.usetex": True
})


np.set_printoptions(precision=2)

In [ ]:
from problem import Problem

from plotting_tools import plot_setup
from simulation import generate_anchors, generate_random_trajectory, generate_distances

np.random.seed(2)
N = 5
K = 3
d = 2
prob = Problem(N=N, K=K, d=d, sigma_acc_est=0.2, sigma_dist_est=0.01)
prob.generate_random(sigma_acc_real=0.2, sigma_dist_real=0.01)
plot_setup(prob.trajectory, prob.anchors)

# Gaussian process estimation

In [ ]:
from plotting_tools import plot_covariance
from gauss_newton import gauss_newton

regularization = "constant-velocity"; k = prob.d * 2
theta_0 = prob.gt_init(regularization)
theta_hat, stats = gauss_newton(theta_0, prob, regularization=regularization)

from scipy.sparse.linalg import inv
K_post_inv = stats["cov"]
K_post = inv(K_post_inv)

scaling = 1e2
fig, ax = plt.subplots()
ax.scatter(*theta_hat[:, :d].T, color="C1", label="measurement points")
for i in range(prob.N):
    x = theta_hat[i, :prob.d]
    plot_covariance(K_post[i*k:i*k+d, i*k:i*k+d].toarray(), x, scaling, ax, facecolor='C2', alpha=0.5)
ax.axis("equal")

# Gaussian process interpolation

## TODO: covariance interpolation is not working yet

In [ ]:
from gaussian_process import query_trajectory, get_prior_covariances


K_prior_ii, K_prior_ij = get_prior_covariances(prob, regularization)
[print(np.linalg.eigvalsh(K)[0]) for K in K_prior_ii]
[print(np.linalg.eigvalsh(K)[0]) for K in K_prior_ij]
K_post_inv = stats["cov"]

print(np.linalg.eigvalsh(K_post.toarray())[0])
np.testing.assert_allclose(inv(K_post).toarray(), K_post_inv.toarray(), rtol=1e-10, atol=1e-10)

fig, ax = plt.subplots()
ax.scatter(*theta_hat[:, :d].T, color="C1", label="measurement points")
ax.scatter(*theta_hat[0, :d], color="C2", marker=".", label="interpolation points")
scaling = 2# heuristic
for tau in np.arange(prob.times[0], prob.times[-1], step=0.1):
    x_tau, K_tau = query_trajectory(tau, prob, theta_hat, regularization=regularization, return_covariance=True,
                                    K_prior_ii=K_prior_ii, K_prior_ij=K_prior_ij, K_post=K_post)
    #print(np.linalg.eigvalsh(K_tau[:d, :d]))
    plot_covariance(K_tau[:d, :d], x_tau[:d], scaling, ax, facecolor='C2', alpha=0.5)
ax.legend()
ax.axis("equal")